In [1]:
import pandas as pd

In [3]:
path = 'Environmental Discourse'

In [17]:
env = pd.read_pickle('../Data/'+path+'/env_0.pkl')
env = env.sample(1000)

In [10]:
env.head()

,source,url,title,date,author,text,year
20339,Resilience,https://www.resilience.org/stories/2008-07-08/...,Asia – July 8,2008-07-08,Staff,. Japan sees a chance to promote its energ...,2008
87817,Grist,https://grist.org/article/the-blame-the-enviro...,A possible smear campaign fingers greens...,2005-09-17,Emily Gertz,The Gonzales Justice Department may be seeking...,2005
63974,Grist,https://grist.org/climate/heres-how-coronaviru...,Here’s how coronavirus affected carbon e...,2020-05-19,Zoya Teirstein,"The pandemic is far from over, but some states...",2020
94514,Grist,https://grist.org/article/2011-01-21-meet-the-...,Meet the Climate Fockers,2011-01-22,Auden Schendler,Here’a repost from my Climateprogress.org blog...,2011
57887,Grist,https://grist.org/living/3-disturbing-facts-pr...,3 disturbing facts prove sexual harassme...,2015-04-16,Eve Andrews,"Hello! On this spring-shiny day, are you think...",2015


# Sentence tokenize

In [21]:
import dask.dataframe as dd
from gensim.utils import effective_n_jobs
import spacy
try:
    nlp = spacy.load("en")
except OSError:
    nlp = spacy.load("en_core_web_sm")

In [15]:
def word_tokenize(word_list, model=nlp, MAX_LEN=1500000):
    
    tokenized = []
    if type(word_list) == list and len(word_list) == 1:
        word_list = word_list[0]

    if type(word_list) == list:
        word_list = ' '.join([str(elem) for elem in word_list]) 
    # since we're only tokenizing, I remove RAM intensive operations and increase max text size

    model.max_length = MAX_LEN
    doc = model(word_list, disable=["parser", "tagger", "ner", "lemmatizer"])
    
    for token in doc:
        if not token.is_punct and len(token.text.strip()) > 0:
            tokenized.append(token.text)
    return tokenized

In [16]:
def normalizeTokens(word_list, extra_stop=[], model=nlp, lemma=True, MAX_LEN=1500000):
    #We can use a generator here as we just need to iterate over it
    normalized = []
    if type(word_list) == list and len(word_list) == 1:
        word_list = word_list[0]

    if type(word_list) == list:
        word_list = ' '.join([str(elem) for elem in word_list]) 

    # since we're only normalizing, I remove RAM intensive operations and increase max text size

    model.max_length = MAX_LEN
    doc = model(word_list.lower(), disable=["parser", "ner"])

    if len(extra_stop) > 0:
        for stopword in extra_stop:
            lexeme = nlp.vocab[stopword]
            lexeme.is_stop = True

    # we check if we want lemmas or not earlier to avoid checking every time we loop
    if lemma:
        for w in doc:
            # if it's not a stop word or punctuation mark, add it to our article
            if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num and len(w.text.strip()) > 0:
            # we add the lematized version of the word
                normalized.append(str(w.lemma_))
    else:
        for w in doc:
            # if it's not a stop word or punctuation mark, add it to our article
            if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num and len(w.text.strip()) > 0:
            # we add the lematized version of the word
                normalized.append(str(w.text.strip()))

    return normalized

In [14]:
def sent_tokenize(word_list, model=nlp):
    doc = model(word_list)
    sentences = [sent.text.strip() for sent in doc.sents]
    return sentences

In [ ]:
d_env = dd.from_pandas(env, npartitions=effective_n_jobs(-1))
env['tokenized_sents'] = env.text.apply(lambda x: [word_tokenize(s) for s in sent_tokenize(x)])
env['normalized_sents'] = env['tokenized_sents'].apply(lambda x: [normalizeTokens(s, lemma=False) for s in x])
env = d_env.compute()

In [19]:
env.shape

(1000, 7)

In [ ]:
d_env = dd.from_pandas(env_tok, npartitions=effective_n_jobs(-1))

In [ ]:
envW2V_skipgram = gensim.models.word2vec.Word2Vec(env['normalized_sents'].sum(), sg=1)